In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path


# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")
MIN_PASSING_SCORE = 70

# Read School and Student Data File and store into Pandas DataFrames

# PyCity Schools Analysis

- Your analysis here
  
---

In [2]:
school_df = pd.read_csv(school_data_to_load)
print(school_df.shape) 
school_df.head()

(15, 5)


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
student_df = pd.read_csv(student_data_to_load)
print(student_df.shape)
student_df.head()

(39170, 7)


,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
# Combine the data into a single dataset.  
complete_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
del student_df
print(complete_df.shape)
complete_df.head()

(39170, 11)


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [5]:
# Calculate the total number of unique schools
school_count = school_df["school_name"].count()
school_count

15

In [6]:
complete_df["school_name"].nunique()

15

In [7]:
# Calculate the total number of students
student_count = complete_df["student_name"].count()
student_count

39170

In [8]:
# Calculate the total budget
total_budget = school_df["budget"].sum()
total_budget

24649428

In [9]:
# Calculate the average (mean) math score
average_math_score = complete_df["math_score"].mean()
average_math_score

78.98537145774827

In [10]:
# Calculate the average (mean) reading score
average_reading_score = complete_df["reading_score"].mean()
average_reading_score

81.87784018381414

In [11]:
passing_math_series=complete_df["math_score"] >= MIN_PASSING_SCORE

In [12]:
complete_df[passing_math_series]["student_name"].count()

29370

In [13]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = complete_df[passing_math_series].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [14]:
passing_reading_series=complete_df["reading_score"] >= MIN_PASSING_SCORE
passing_reading_series.head()

0    False
1     True
2     True
3    False
4     True
Name: reading_score, dtype: bool

In [15]:
complete_df[passing_reading_series]["student_name"].count()

33610

In [16]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = complete_df[passing_reading_series].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [17]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = complete_df.loc[passing_math_series&passing_reading_series, "student_name"].count()
passing_math_reading_percentage = 100*(passing_math_reading_count / student_count) 
passing_math_reading_percentage

65.17232575950983

In [18]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
summary_data = {
    "Total Schools": school_count,
    "Total Students": student_count,
    "Total Budget": total_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Math": passing_math_percentage,
    "% Passing Reading": passing_reading_percentage,
    "% Overall Passing": passing_math_reading_percentage,
    
}

In [19]:
district_summary = pd.DataFrame(summary_data, index=[0])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [32]:
# Use the code provided to select all of the school type
school_types = school_df.set_index(['school_name'])['type']
school_types.head()

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Name: type, dtype: object

In [33]:
# Calculate the total student count per school
per_school_counts = complete_df["school_name"].value_counts()
per_school_counts.head()

school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Name: count, dtype: int64

In [34]:
per_school_budget = complete_df.groupby("school_name")["budget"].mean()

In [35]:
# Calculate the total school budget and per capita spending per school
per_student_budget = per_school_budget/per_school_counts
per_student_budget.head()

school_name
Bailey High School      628.0
Cabrera High School     582.0
Figueroa High School    639.0
Ford High School        644.0
Griffin High School     625.0
dtype: float64

In [36]:
# Calculate the average test scores per school
per_school_math = complete_df.groupby("school_name")["math_score"].mean()
per_school_reading = complete_df.groupby("school_name")["reading_score"].mean()

In [37]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = complete_df[complete_df["math_score"] >=70]
students_passing_math_counts = students_passing_math.groupby("school_name").size()

In [38]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = complete_df[complete_df["reading_score"] >=70]
students_passing_reading_counts = students_passing_reading.groupby("school_name").size()

In [39]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = complete_df[(complete_df['reading_score'] >= 70) & (complete_df['math_score'] >= 70)].groupby('school_name').size()
students_passing_math_and_reading .head()

school_name
Bailey High School      2719
Cabrera High School     1697
Figueroa High School    1569
Ford High School        1487
Griffin High School     1330
dtype: int64

In [50]:
# Use the provided code to calculate the passing rates
per_school_passing_math = students_passing_math_counts / per_school_counts * 100
per_school_passing_reading = students_passing_reading_counts / per_school_counts * 100
overall_passing_rate = students_passing_math_and_reading / per_school_counts * 100


In [52]:
per_school_summary = pd.DataFrame({
    "School Types": school_types,
    "Total Schools": per_school_counts,
    "Per Student Budget": per_student_budget,
    "Total School Budget": per_school_budget,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": overall_passing_rate})

#Formatting
per_school_summary["Total School Budget"] = per_school_summary.apply(lambda x: "{:,.0f}".format(x["Total School Budget"]), axis=1)
per_school_summary["Per Student Budget"] = per_school_summary.apply(lambda x: "{:,.0f}".format(x["Per Student Budget"]), axis=1)

#Display the DataFrame
per_school_summary

,School Types,Total Schools,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,628,"3,124,928",77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,582,"1,081,356",83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,639,"1,884,411",76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,644,"1,763,916",77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,625,"917,500",83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,652,"3,022,020",77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,581,"248,087",83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,655,"1,910,635",76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,650,"3,094,650",77.072464,80.966394,66.057551,81.222432,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [53]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False).head(5).reset_index(drop=True)

top_schools.head(5)

,School Types,Total Schools,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Charter,1858,582,"1,081,356",83.061895,83.975780,94.133477,97.039828,91.334769
1,Charter,1635,638,"1,043,130",83.418349,83.848930,93.272171,97.308869,90.948012
2,Charter,1468,625,"917,500",83.351499,83.816757,93.392371,97.138965,90.599455
3,Charter,2283,578,"1,319,574",83.274201,83.989488,93.867718,96.539641,90.582567
4,Charter,962,609,"585,858",83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [54]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=True).head(5).reset_index(drop=True)
bottom_schools.head(5)


,School Types,Total Schools,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,District,3999,637,"2,547,363",76.842711,80.744686,66.366592,80.220055,52.988247
1,District,2949,639,"1,884,411",76.711767,81.158020,65.988471,80.739234,53.204476
2,District,2917,655,"1,910,635",76.629414,81.182722,65.683922,81.316421,53.513884
3,District,4635,652,"3,022,020",77.289752,80.934412,66.752967,80.862999,53.527508
4,District,4761,650,"3,094,650",77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [59]:
# Use the code provided to separate the data by grade
ninth_graders = complete_df[(complete_df["grade"] == "9th")]
tenth_graders = complete_df[(complete_df["grade"] == "10th")]
eleventh_graders = complete_df[(complete_df["grade"] == "11th")]
twelfth_graders = complete_df[(complete_df["grade"] == "12th")]

In [60]:
# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = 
tenth_grader_math_scores = 
eleventh_grader_math_scores = 
twelfth_grader_math_scores = 

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = 

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

SyntaxError: invalid syntax (679687903.py, line 2)

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = 
tenth_grader_reading_scores = 
eleventh_grader_reading_scores = 
twelfth_grader_reading_scores = 

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = 

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_summary["Per Student Budget"], spending_bins, labels=group_names)
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary